<a href="https://colab.research.google.com/github/sominshim/Predicting_Personality_through_Text/blob/Ujin/mbti_crawling_kor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 4.2MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:14 http://p

In [11]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from pandas import Series, DataFrame
import pandas as pd
import time
import re
from tqdm import tqdm_notebook

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


In [5]:
USER = 'ID'
PASS = 'password'

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

#로그인 페이지 접근

url_login = "https://nid.naver.com/nidlogin.login"

wd.get(url_login)

#텍스트박스에 아이디, 비밀번호 입력하기

e = wd.find_element_by_id("id")

e.clear()

e.send_keys(USER)

e = wd.find_element_by_id("pw")

e.clear()

e.send_keys(PASS)


#입력 양식 전송, 로그인

form = wd.find_element_by_css_selector("input.btn_global[type=submit]")

form.submit()

#print("네이버에 로그인...")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [6]:
categories = ['ISTJ', 'ISFJ', 'ESTJ', 'ESFJ', 'ISTP', 'ESTP', 'ESFP','ISFP', 
              'INTJ','INTP','ENTJ', 'ENTP', 'INFJ','INFP','ENFJ','ENFP']

In [12]:
def crawler(url_sub):
  contents = []
  names = []
  for link in tqdm_notebook(url_sub):
    try:
      # print(link)
      wd.get(link)
      time.sleep(1)
      wd.switch_to.frame('cafe_main')
      time.sleep(1)
      soup = BeautifulSoup(wd.page_source, 'lxml')

      nameArea = soup.find('div', {'class':'nick_box'})
      name_parags = nameArea.findAll('a')  
      # name = ""
      for parag in name_parags:
        t = parag.text.strip()
        text = re.sub('[^a-zA-Z]','',t.upper())
      if (categories[0] in text)|(categories[1] in text)|(categories[2] in text)|(categories[3] in text)|(categories[4] in text)|(categories[5] in text)|(categories[6] in text)|(categories[7] in text)|(categories[8] in text)|(categories[9] in text)|(categories[10] in text)|(categories[11] in text)|(categories[12] in text)|(categories[13] in text)|(categories[14] in text)|(categories[15] in text):
        # print(text)
        names.append(text)
      else: continue

      contentArea = soup.find('div', {'class':'article_viewer'})
      parags = contentArea.findAll('p')  
      content = ""
      for parag in parags:
          content += parag.text.strip()
      contents.append(content)

      pass
    except:
      # Continue to next iteration.
      continue

  return contents, names

In [ ]:
iter = 1
for i in range(500, 1000, 1):
  # print(i)
  # url = mbti_url + str(i+1)
  wd.get('https://cafe.naver.com/ArticleList.nhn?search.clubid=11856775&search.boardtype=L&search.totalCount=151&search.page='+ str(i+1))  #mbti카페
  time.sleep(1)
  wd.switch_to.frame('cafe_main')  #카페 메인 아이프레임
  html = wd.page_source  #페이지 소스 가져오기
  soup = BeautifulSoup(html, "html.parser")

  title = soup.select('.inner_list')
  url_sub = [] # 각 게시물 url 뽑음

  for link in title:
    linkadd = link.find("a")['href']
    url_sub.append("https://cafe.naver.com" + linkadd)
    
  df1, df2 = crawler(url_sub)
  df1 = pd.DataFrame(data = df1, columns=['posts'])
  df2 = pd.DataFrame(data = df2, columns=['type'])
  add = pd.concat([df2, df1], axis=1)

  if i == 501:
    df = add
  else:
    df = pd.concat([df, add], axis=0)


In [14]:
df

,type,posts
0,INTPTW,주제 : 아무거나
1,INFP,정말 친한 동성친구한테 어떻게 대하시나요?​저는 인프피고 친구는 잇프피인데 전 정말...
2,ESFJW,내일 관심남 대학원 발표 나는데​내가 떨어지면 전화하라고 위로해준다 했는데남성분이 ...
3,ISFJ,너무 너무 궁금해서 엠헬 분들 분석(?)이 궁금해요!​김소연 씨와 이상우 씨는 한 ...
4,ENTPWSPSX,실제 창작물에서도 자기 존재 잘감추는 캐릭터들은 하나같이 인프제이기도 하고요 ㅋㅋ​...
...,...,...
2,ENTP,1. 당신에게 고민을 말한다- 엔팁은 고민 말하면 약점 잡히는거 같고 자존심 상해서...
3,INFP,이름: 김아별 (에니어 9w1)​1. 순진하다. 남을 너무 잘 믿어서 바보 취급 받...
4,ENTP,.............?뭐야.사랑하면 닮는다더니.직관 그 자체​​​​​​당분간 인...
5,ENFJ,이게 무슨 유형일진 모르겠지만!눈 앞에 바로 얻을 수 있는 음식이란 보상보다나중에 ...


In [15]:
df.to_csv('/content/drive/My Drive/mbti_crawling_kor.csv')